# Forecast

## Setup

In [ ]:
import sys
sys.path.append("..")

# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy as sp

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, ETS, Naive

from pathlib import Path
from joblib import Parallel, delayed
from itertools import product
from tqdm import tqdm

# Library settings
pd.options.display.max_columns = 999
plt.rcParams["figure.figsize"] = (16, 4)


## Prepare data

In [ ]:
sales = pd.read_csv("../data/sales_train_evaluation.csv")
sales.head()


In [ ]:
calendar = pd.read_csv("../data/calendar.csv", parse_dates=["date"])
calendar.head()

In [ ]:
sell_prices = pd.read_csv("../data/sell_prices.csv")
sell_prices.head()

In [ ]:
def prepare_data(sales, calendar, sell_prices, freq=None):
    Path("../data/processed").mkdir(exist_ok=True)

    hierarchy_df = sales.iloc[:, 1:6].copy()
    df = sales.drop(columns=["id", "dept_id", "cat_id", "state_id"])
    df = df.reset_index().rename(columns={"index": "unique_id"})
    df_long = pd.melt(df, id_vars=df.columns[:3], var_name="d", value_name="sales")
    df_long = df_long.merge(calendar[["date", "d", "wm_yr_wk"]], on="d", how="left")
    df_long = df_long.merge(sell_prices, on=["item_id", "store_id", "wm_yr_wk"], how="left")
    df_long["dollar_sales"] = df_long["sales"] * df_long["sell_price"]
    df_long.loc[df_long.sales == 0, "dollar_sales"] = 0
    df_long["d"] = df_long["d"].str[2:].astype("int")
    df = pd.pivot_table(df_long, values=["sales", "dollar_sales"], index=["date"], columns=["unique_id"])

    if freq is not None:
        df = df.resample(freq).sum()

    bts_df = df["sales"].reset_index()
    bts_df.columns = bts_df.columns.astype("str")

    bts_dollar_df = df["dollar_sales"].reset_index()
    bts_dollar_df.columns = bts_dollar_df.columns.astype("str")

    filepath = Path("../data/processed/bts.csv")
    bts_df.to_csv(filepath, index=False)
    print(f"Data written to {filepath}")
    
    filepath = Path("../data/processed/bts_dollar.csv")
    bts_dollar_df.to_csv(filepath, index=False)
    print(f"Data written to {filepath}")

    filepath = Path("../data/processed/hierarchy.csv")
    hierarchy_df.to_csv(filepath, index=False)
    print(f"Data written to {filepath}")
    
    return bts_df, bts_dollar_df, hierarchy_df

In [ ]:
bts_df, bts_dollar_df, hierarchy_df = prepare_data(sales, calendar, sell_prices, freq="MS")

In [ ]:
bts_df

In [ ]:
bts_dollar_df

In [ ]:
hierarchy_df

## Build S matrix

In [ ]:
# hierarchy_df = pd.read_csv("../data/processed/hierarchy.csv")

In [ ]:
agg_levels = (
    tuple(["TOTAL"]),
    tuple(['state_id']),
    tuple(['store_id']),
    tuple(['cat_id']),
    tuple(['dept_id']),
    tuple(['state_id', 'cat_id']),
    tuple(['state_id', 'dept_id']),
    tuple(['store_id', 'cat_id']),
    tuple(['store_id', 'dept_id']),
    tuple(['item_id']),
    tuple(['item_id', 'state_id']),
    tuple(['item_id', 'store_id']),
)

In [ ]:
def generate_Smatrix(hierarchy_df, agg_levels, sparse=True):
    arr = np.array
    eye = np.eye
    stack = np.row_stack
    
    if sparse:
        arr = sp.sparse.csr_array
        eye = sp.sparse.eye
        stack = sp.sparse.vstack

    def build_row(hierarchy_df, level, comb):
        row = (hierarchy_df.loc[:, level] == comb).apply(lambda x: all(x), axis=1).astype("int16")
        return row
    
    top_row = np.ones(len(hierarchy_df))
    S_rows = [top_row]

    for level in agg_levels[1:-1]:
        print(level)
        combinations = hierarchy_df.loc[:, level].drop_duplicates().to_numpy()
        rows = Parallel(n_jobs=8)(
            delayed(build_row)(hierarchy_df, level, comb) for comb in combinations
        )
        S_rows.extend(rows)

    S_top = arr(S_rows)
    S_bottom = eye(len(hierarchy_df))
    S_arr = stack([S_top, S_bottom])
    
    filepath = Path("../data/processed/S_arr.npz")
    sp.sparse.save_npz(filepath, S_arr)
    print(f"Data written to {filepath}")
    return S_arr

In [ ]:
S_arr = generate_Smatrix(hierarchy_df, agg_levels)

In [ ]:
S_arr.shape

## Generate all the timeseries from the hierarchy

In [ ]:
bts_df = pd.read_csv("../data/processed/bts.csv", parse_dates=["date"])
bts_dollar_df = pd.read_csv("../data/processed/bts_dollar.csv", parse_dates=["date"])
S_arr = sp.sparse.load_npz("../data/processed/S_arr.npz")

In [ ]:
def generate_yts(S_arr, bts_df):
    bts_arr = bts_df.set_index("date").to_numpy()
    yts_arr = S_arr @ bts_arr.T
    yts_df = pd.DataFrame(yts_arr.T)
    yts_df.insert(0, "date", bts_df["date"])
    return yts_df

yts_df = generate_yts(S_arr, bts_df)
yts_dollar_df = generate_yts(S_arr, bts_dollar_df)

filepath = Path("../data/processed/yts.csv")
yts_df.to_csv(filepath, index=False)
print(f"Data written to {filepath}")

filepath = Path("../data/processed/yts_dollar.csv")
yts_dollar_df.to_csv(filepath, index=False)
print(f"Data written to {filepath}")

In [ ]:
yts_df

In [ ]:
yts_dollar_df

## Create panel data

In [ ]:
yts_df = pd.read_csv("../data/processed/yts.csv", parse_dates=["date"])

In [ ]:
def generate_panel_df(yts_df):
    y_list = []
    for col in tqdm(yts_df.columns[1:]):
        y = yts_df[["date", col]]
        y = y.rename(columns={"date": "ds", col: "y"})
        y.insert(0, "unique_id", col)
        y_list.append(y)
    yts_panel = pd.concat(y_list)

    filepath = Path("../data/processed/yts_panel.csv")
    yts_panel.to_csv(filepath, index=False)
    print(f"Data written to {filepath}")

    return yts_panel

In [ ]:
yts_panel = generate_panel_df(yts_df)

## Forecast

In [ ]:
# yts_panel = pd.read_csv("../data/processed/yts_panel.csv", parse_dates=["ds"])

In [ ]:
# We burn the first and last month because they had less data during resampling
yts_panel = yts_panel[(yts_panel.ds > "2011-01-01") & (yts_panel.ds < "2016-05-01")]

In [ ]:
# Smaller dataset to test the pipeline
yts_panel = yts_panel[yts_panel.unique_id < 100]

In [ ]:
yts_train = yts_panel[(yts_panel.ds <= "2015-04-01")]
yts_test = yts_panel[(yts_panel.ds > "2015-04-01")]

In [ ]:
i = 0
yts_train.loc[yts_train.unique_id == i].set_index("ds")["y"].plot()
yts_test.loc[yts_test.unique_id == i].set_index("ds")["y"].plot()

In [ ]:
models = [
    ETS(season_length=12, model='ZZA'),
    AutoARIMA(season_length=12, max_d=1, max_D=1)
]

model = StatsForecast(
    df=yts_train, 
    models=models,
    freq='MS',
    fallback_model=Naive(),
    n_jobs=-1,
)

fcst_df = model.forecast(12, level=[80, 95], fitted=True).reset_index()
fitted_df = model.forecast_fitted_values().reset_index()

Path("../fcst").mkdir(exist_ok=True)

filepath = Path("../fcst/fcst.csv")
fcst_df.to_csv(filepath, index=False)
print(f"Data written to {filepath}")

filepath = Path("../fcst/fitted.csv")
fitted_df.to_csv(filepath, index=False)
print(f"Data written to {filepath}")

## Plot results

In [ ]:
#fcst_df = pd.read_csv("../fcst/fcst.csv", parse_dates=["ds"])
#fitted_df = pd.read_csv("../fcst/fitted.csv", parse_dates=["ds"])

In [ ]:
i = 50

yts_train.loc[yts_train.unique_id == i].set_index("ds")["y"].plot()
yts_test.loc[yts_test.unique_id == i].set_index("ds")["y"].plot()

fitted_df.loc[fitted_df.unique_id == i].set_index("ds")["ETS"].plot(style=":")
fitted_df.loc[fitted_df.unique_id == i].set_index("ds")["AutoARIMA"].plot(style=":")

fcst_df.loc[fcst_df.unique_id == i].set_index("ds")["ETS"].plot(style="--")
fcst_df.loc[fcst_df.unique_id == i].set_index("ds")["AutoARIMA"].plot(style="--")

plt.legend()